In [290]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from scipy import sparse

In [7]:
items = pd.read_csv('data/item_to_id.csv')

In [8]:
purch = pd.read_csv('data/purchase_history.csv')

In [14]:
items.shape, items.head()


((48, 2),          Item_name  Item_id
 0           coffee       43
 1              tea       23
 2            juice       38
 3             soda        9
 4  sandwich loaves       39)

In [298]:
item_dict = dict(zip(items.Item_id,items.Item_name))

In [299]:
item_dict

{43: 'coffee',
 23: 'tea',
 38: 'juice',
 9: 'soda',
 39: 'sandwich loaves',
 37: 'dinner rolls',
 34: 'tortillas',
 13: 'bagels',
 28: 'canned vegetables',
 26: 'spaghetti sauce',
 41: 'ketchup',
 21: 'cheeses',
 14: 'eggs',
 16: 'milk',
 48: 'yogurt',
 8: 'butter',
 11: 'cereals',
 30: 'flour',
 1: 'sugar',
 31: 'pasta',
 5: 'waffles',
 22: 'frozen vegetables',
 36: 'ice cream',
 6: 'poultry',
 17: 'beef',
 47: 'pork',
 46: 'bananas',
 40: 'berries',
 25: 'cherries',
 20: 'grapefruit',
 32: 'apples',
 44: 'broccoli',
 10: 'carrots',
 45: 'cauliflower',
 42: 'cucumbers',
 2: 'lettuce',
 18: 'laundry detergent',
 27: 'dishwashing\xa0',
 24: 'paper towels',
 33: 'toilet paper',
 15: 'aluminum foil',
 7: 'sandwich bags',
 12: 'shampoo',
 35: 'soap',
 29: 'hand soap',
 19: 'shaving cream',
 4: 'baby items',
 3: 'pet items'}

In [15]:
purch.shape, purch.head()


((39474, 2),    user_id                                                 id
 0   222087                                              27,26
 1  1343649                                            6,47,17
 2   404134                       18,12,23,22,27,43,38,20,35,1
 3  1110200                                 9,23,2,20,26,47,37
 4   224107  31,18,5,13,1,21,48,16,26,2,44,32,20,37,42,35,4...)

In [22]:
len(purch.user_id.unique())

24885

In [18]:
purch.user_id.value_counts()

696000     7
31625      7
653800     7
884172     7
315516     6
453019     6
38872      6
779061     6
1374100    6
813270     6
64998      6
1271578    6
1499087    6
917199     6
946453     6
373484     6
1270373    6
367872     6
599172     6
238495     6
617988     6
719358     6
117178     6
920002     6
269335     6
723012     6
1485538    6
828721     6
820788     5
1494796    5
          ..
33986      1
187593     1
785613     1
295767     1
1254614    1
641290     1
43310      1
744665     1
627936     1
961767     1
512403     1
1195253    1
1328314    1
875705     1
939192     1
486355     1
736730     1
1240239    1
1092775    1
365728     1
1076383    1
243198     1
1193108    1
894126     1
1485489    1
916621     1
652428     1
840840     1
1401982    1
1023836    1
Name: user_id, Length: 24885, dtype: int64

In [61]:
purch['item_count']=purch['id'].count()

In [86]:
iz=purch[:1]['id'].values

In [104]:
item_counts=[]
customer_items=defaultdict()
for idx in purch['id']:
    item_counts.append(len(idx))
    

In [108]:
purch['item_counts']=item_counts

In [113]:
purch[purch['item_counts']==purch['item_counts'].max()]

,user_id,id,item_count,item_counts
3332,388149,"18,38,36,7,1,10,45,26,39,5,44,40,32,25,9,21,28...",39474,75


# the customer who bought the most items overall in her lifetime

In [125]:
purch.groupby('user_id')['item_counts'].sum().reset_index().sort_values('item_counts', ascending=False).head(1)


,user_id,item_counts
4445,269335,194


In [126]:
purch[purch['user_id']==269335]

,user_id,id,item_count,item_counts
12030,269335,"2,1,42,11,9,39,37,19,38,23",39474,26
21971,269335,"46,14,2,24,4,18,9,6,10,41,28,42,44,23",39474,37
25662,269335,"13,47,21,2,9,23,36,48,17,42,10,37,6",39474,35
31105,269335,"37,13,6,2,18,3,42,9,38,35,24,36,39",39474,34
34939,269335,"44,3,11,22,2,10,36,25,30,40,28,20,41",39474,36
38512,269335,"17,12,21,37,34,36,43,26,28",39474,26


In [177]:
matrix_dict=defaultdict(int)

In [178]:
for i in range(purch.shape[0]):
    #print('i',purch.user_id[i])
    for item in purch['id'][i].split(','):
        matrix_dict[purch.user_id[i],int(item)]+=1

In [179]:
len(matrix_dict)


290558

In [180]:
rows, cols, vals = [], [], []
for key, value in matrix_dict.items():
    #print(key[0])
    rows.append(key[0])
    cols.append(key[1])
    vals.append(matrix_dict[key])
sparse_matrix = sparse.csr_matrix((vals, (rows, cols)))

In [227]:
dense = sparse_matrix.todense()

In [231]:
df = pd.DataFrame(dense)

In [237]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# for each item, the customer who bought that product the most

In [236]:
df.idxmax(axis=0)

0           0
1       31625
2       31625
3      154960
4        5289
5      217277
6      334664
7      175865
8      151926
9      269335
10     618914
11     367872
12     557904
13     653800
14     172120
15     143741
16      73071
17     366155
18     917199
19      31625
20     885474
21     884172
22    1199670
23     920002
24     189913
25      68282
26     967573
27     956666
28     204624
29     394348
30      21779
31     289360
32     109578
33    1310207
34     305916
35     450482
36     269335
37      46757
38     255546
39     599172
40      38872
41     133355
42      80215
43     996380
44      31625
45    1198106
46    1218645
47     384935
48     335841
dtype: int64

In [238]:
def fit_NMF(sparse_matrix_,n_components_,max_iter=250):
    '''This function takes a sparse matrix, the number of components (topics) and max iterations allowed'''
    from sklearn.decomposition import NMF
    model = NMF(n_components=n_components_,max_iter=max_iter)
    W = model.fit_transform(sparse_matrix_) # (transactions, topics)
    H=model.components_ # (topics,items)
    model_iter = model.n_iter_ 
    #display at console the number of iterations and the shape of the decomposed matrices
    print('iterations:',model_iter,'W shape:',W.shape,'H shape:',H.shape)
    
    #calculate the strength of each topic my making it a hard classifier and scoring each topic accordingly
    #make w a matrix of zeros shaped like W
    w = np.zeros_like(W)
    #if its the strongest, give it a 1!
    w[np.arange(len(W)), W.argmax(1)] = 1
    topic_strength = np.sum(w,axis=0)
    topic_strength = np.round(topic_strength/topic_strength.sum(),2)
    for i,t in enumerate(topic_strength):
        print('topic %d strength: %f '%(i,t))
    
    return model,W,H,model_iter,topic_strength

In [285]:
items_set=items.Item_name.unique()

In [286]:
len(items_set)

48

In [288]:
n_top_words=10
number_of_components=4

In [289]:
model,W,H,model_iter,topic_strength = fit_NMF(sparse_matrix,n_components_=number_of_components)

iterations: 217 W shape: (1499975, 4) H shape: (4, 49)
topic 0 strength: 0.990000 
topic 1 strength: 0.000000 
topic 2 strength: 0.000000 
topic 3 strength: 0.000000 


In [302]:
def print_top_items(model, feature_names, n_top_words):
    '''This fuction requires a model, a list of items, and the number of top words per topic
    to create words clouds and lists of items for the n_top_words in each topic'''
    topic_dict = defaultdict()
    topics_list =[]
    fig = plt.figure(1,figsize=(100,100))
    #itereate through each topic and create words clouds
    for topic_idx, topic in enumerate(model.components_):
        print(topic_strength[topic_idx],"Topic #%d of %d:" %( topic_idx,number_of_components))
        #topic_string=(" ".join([feature_names[i]for i in topic.argsort()[:-n_top_words - 1:-1]]))
        topic_string=[feature_names[i]for i in topic.argsort()[:-n_top_words-1:-1]-1]
        topic_dict[topic_idx]=topic_string
        topics_list.append(topic_string)
        print(topic_string)
        '''
        #create wordclouds, remove spaces between words
        wordcloud = WordCloud(max_font_size=500, max_words=1000, background_color="white").generate(str(topic_string).replace(" ", '').replace("'",""))
        #wordcloud = WordCloud(max_font_size=500, max_words=1000, background_color="white").generate(str(topic_string).replace("'","").replace(",",""))
        ax = fig.add_subplot(1,number_of_components,topic_idx+1)
        ax.imshow(wordcloud)
        ax.axis("off")
        # Display the generated image:
        #plt.figure(1,figsize=(10,10))
        
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
        
        #plt.savefig('topic%d.png'%topic_idx)
        #plt.close()
        img_time=str(time.time()).split('.')[0]
        wordcloud.to_file('./img/2018/%s.topic%d.png'%(img_time,topic_idx))
        print('<img src="./img/2018/%s.topic%d.png">'%(img_time,topic_idx))
        '''
    #create a matrix to compare the similarities of different topics top words
    topic_compare = np.zeros([len(topic_dict),len(topic_dict)])
    #fill the topic_compare matrix with count of similar words to other matricies
    for topic in topic_dict:
        for item in topic_dict[topic]:
            for topic2 in topic_dict:
                if item in topic_dict[topic2]:
                    topic_compare[topic,topic2]+=1
    #display the results
    print ('topic similarity matix:')
    print (topic_strength)            
    print(topic_compare)

    topic_matrix=np.array(topics_list).T
    print (topic_matrix.shape)
    topic_df = pd.DataFrame(topic_matrix,columns=topic_strength)
    #create markdown friendly dataframe of top items per topic
    import tabulate 
    print(tabulate.tabulate(topic_df.values,topic_df.columns, tablefmt="pipe")) 
    return topic_dict,topics_list
    

In [303]:
topic_dict,topics_list = print_top_items(model,list(items_set),n_top_words)

0.99 Topic #0 of 4:
['tea', 'sandwich bags', 'soap', 'spaghetti sauce', 'hand soap', 'coffee', 'juice', 'cereals', 'dinner rolls', 'baby items']
0.0 Topic #1 of 4:
['beef', 'pasta', 'broccoli', 'shaving cream', 'toilet paper', 'baby items', 'juice', 'dinner rolls', 'bananas', 'frozen vegetables']
0.0 Topic #2 of 4:
['cauliflower', 'paper towels', 'eggs', 'laundry detergent', 'butter', 'coffee', 'bagels', 'waffles', 'milk', 'pet items']
0.0 Topic #3 of 4:
['canned vegetables', 'dishwashing\xa0', 'shampoo', 'ice cream', 'dinner rolls', 'cereals', 'baby items', 'coffee', 'pork', 'bananas']
topic similarity matix:
[0.99 0.   0.   0.  ]
[[10.  3.  1.  4.]
 [ 3. 10.  0.  3.]
 [ 1.  0. 10.  1.]
 [ 4.  3.  1. 10.]]
(10, 4)
| 0.99            | 0.0               | 0.0               | 0.0               |
|:----------------|:------------------|:------------------|:------------------|
| tea             | beef              | cauliflower       | canned vegetables |
| sandwich bags   | pasta          

<Figure size 7200x7200 with 0 Axes>

In [215]:
k

5848

In [217]:
sparse_matrix.max(axis=0)

<1x49 sparse matrix of type '<class 'numpy.int64'>'
	with 48 stored elements in COOrdinate format>